In [ ]:
import kratos
from agno.agent import Agent
from agno.models.lmstudio import LMStudio
from agno.tools.youtube import YouTubeTools
from agno.tools.yfinance import YFinanceTools

# Creating a few agents using the Agno framework for building agents
# All the llm models here use LMStudio as the provider and connect to local API server
# I'm using qwen/qwen3-4b model for both agents, but you can use any other model that you have access to.

finance_agent = Agent(
    model=LMStudio(
        # id="qwen/qwen3-4b",
        id="qwen/qwen3-4b",
        base_url="http://host.docker.internal:1234/v1",
    ),
    name="KratosFinance",
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions="You are Kratos Finance Agent, specialized in stock market analysis and financial data. Use YFinance to get current stock prices, analyst recommendations, and company information. When users ask about stocks, always provide current prices and relevant financial metrics. Be precise with numbers and explain what the data means for investment decisions.",
)


youtube_agent = Agent(
    model=LMStudio(
        id="qwen/qwen3-4b",
        base_url="http://host.docker.internal:1234/v1",
    ),
    name="KratosYouTube",
    tools=[YouTubeTools()],
    instructions="You are Kratos YouTube Agent, specialized in analyzing YouTube videos and transcripts. Use YouTube tools to extract video information, transcripts, and analyze content. When users mention YouTube videos, provide detailed analysis of the content, key points, and insights from the transcript. Be thorough in your analysis and highlight important information.",
)


In [2]:
# Submit these agents so that they are ready to be used at a later point - none of
# these models are actually loaded in memory right now and thus no resources are wasted.

print(kratos.submit(finance_agent, name="agent-finance", dependencies=["yfinance"]))
print(kratos.submit(youtube_agent, name="agent-youtube", dependencies=["youtube-transcript-api"]))

agent-finance
agent-youtube


In [3]:
# The name parameter we used in submit() acts as an identifier as to which agent do you want to invoke.

kratos.invoke(name="agent-finance", task="How has the trend been for NVIDIA stock?")

<think>
Okay, the user is asking about the trend for NVIDIA stock. First, I need to figure out what exactly they mean by "trend." Typically, when someone asks about a stock's trend, they're interested in its recent performance, maybe over the past few days or weeks. But since I don't have a function to get historical price data, I can only provide the current price and maybe some analyst insights.
Wait, the tools available are get_current_stock_price, get_company_info, and get_analyst_recommendations. So I can't fetch historical data directly. But maybe I can start by getting the current price to give them a snapshot. Also, analyst recommendations might give some insight into future expectations. Additionally, company information could explain factors affecting the stock.
But the user specifically asked about the trend, which implies a direction over time. Since I can't provide historical data, I should inform them that I can't retrieve past prices but can offer the latest price and an

In [4]:
kratos.invoke(name="agent-youtube", task="Summarize the transcript of this video: https://www.youtube.com/watch?v=dQw4w9WgXcQ")

<think>
Okay, the user wants me to summarize the transcript of the YouTube video with the URL https://www.youtube.com/watch?v=dQw4w9WgXcQ. Let me figure out how to approach this.
First, I need to get the transcript from the video. The available tools include get_youtube_video_captions, which is designed to retrieve captions from a YouTube video. The function requires the video URL, so I should call that first. Once I have the captions, I can then process them to create a summary. 
Wait, the user mentioned the transcript, so maybe the captions are in the form of a transcript. The get_youtube_video_captions function should fetch the captions, which are usually in a text format. After retrieving the captions, I can analyze the text to summarize the key points. 
But I need to make sure that the function is called correctly. The URL provided is valid, so I'll use that. Let me check the parameters for get_youtube_video_captions: it takes the video URL as a parameter. So I'll structure the to

In [5]:
# from concurrent.futures import ThreadPoolExecutor, wait

# print(f"\n🚀 Starting 3 concurrent invocations where each instance of an agent will have it's own model loaded in memory")

# tasks = [
#     "Stock price of NVIDIA",
#     "Stock price of Apple",
#     "Stock price of Tesla",
# ]

# with ThreadPoolExecutor(max_workers=3) as executor:
#     futures = [executor.submit(kratos.invoke, 'agent-finance', task, False) for task in tasks]

#     print(f"\n🔄 Waiting for all agents to complete...")
#     wait(futures)

#     print(f"\n🔄 All agents have completed their tasks")

# print("\n\nQ&A:")